In [ ]:
import os
import pandas as pd
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load the Sikka API CSV file
csv_path = "/Users/piyushpatil/Desktop/Piyush/Projects/AI Agent /Updated_APIs.csv"
df = pd.read_csv(csv_path)

# Initialize SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings from API descriptions
descriptions = df["Description"].astype(str).tolist()
embeddings = model.encode(descriptions).astype("float32")

# Create FAISS index and add embeddings
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Define writable output paths
faiss_index_path = "/tmp/api_index.faiss"
metadata_path = "/tmp/api_metadata.pkl"

# Save FAISS index
faiss.write_index(index, faiss_index_path)

# Save API metadata
metadata = df.to_dict(orient="records")
with open(metadata_path, "wb") as f:
    pickle.dump(metadata, f)

print(f"✅ FAISS index and metadata saved!\n- Index: {faiss_index_path}\n- Metadata: {metadata_path}")


✅ FAISS index and metadata saved!
- Index: /tmp/sikka_api_index.faiss
- Metadata: /tmp/sikka_api_metadata.pkl


In [ ]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index
index = faiss.read_index("/tmp/api_index.faiss")

# Load metadata (API name, description, parameters)
with open("/tmp/sikka_api_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

def search_sikka_api(query: str, k: int = 5) -> list:
    """Search top matching APIs using semantic FAISS index."""
    query_embedding = model.encode([query]).astype("float32")
    D, I = index.search(query_embedding, k)
    query_tokens = query.lower().split()

    results = []
    for idx, score in zip(I[0], D[0]):
        entry = metadata[idx]
        param_list = [p.strip() for p in str(entry.get("Parameters", "")).split(",") if p.strip()]
        matched = [p for p in param_list if any(token in p.lower() for token in query_tokens)]

        results.append({
            "api_name": entry.get("API Name", "N/A"),
            "description": entry.get("Description", "No description"),
            "endpoint": entry.get("API Endpoints", "N/A"),
            "similarity_score": float(score),
            "parameters": param_list,
            "matched_parameters": matched or ["No direct match"]
        })
    return results

def format_api_results(api_list: list) -> str:
    """Format API results into a readable string for CrewAI agents, with endpoint at top."""
    output = ""
    for i, api in enumerate(api_list, 1):
        output += f"""
🔹 API #{i}
🔗 Endpoint: {api['endpoint']}  <-- Moved this to top!
📛 Name: {api['api_name']}
📄 Description: {api['description']}
📦 Parameters: {', '.join(api['parameters'])}
🎯 Matched Parameters: {', '.join(api['matched_parameters'])}
------------------------------------------------------------\n"""
    return output.strip()

In [14]:
%pip install python-dotenv
import os
from dotenv import load_dotenv

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
from dotenv import load_dotenv

load_dotenv()  # ✅ Load variables from .env

api_key = os.getenv("OPENAI_API_KEY")   # Load the OpenAI API key from .env file
# Check if the API key is loaded                    

if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file.")
else:
    print("✅ API key loaded successfully.")


✅ API key loaded successfully.


In [ ]:
# Keep the same imports you had
import os
import re
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
%pip install faiss-cpu
# from faiss_search import search_sikka_api, format_api_results
 
# Improved extract_code function
def extract_code(output: str, keyword: str) -> str:
    language_aliases = {
        "FastAPI": ["python", "fastapi", "py"],
        "React": ["jsx", "react", "javascript", "tsx", "js", "ts"],
        "SQL": ["sql", "postgresql", "mysql"],
        "Shell": ["bash", "shell", "sh", "zsh"]
    }
 
    aliases = language_aliases.get(keyword, [keyword])
    pattern = "|".join([re.escape(alias.lower()) for alias in aliases])
    regex = rf"```(?:{pattern})?\s*(.*?)```"
 
    code_blocks = re.findall(regex, str(output), re.DOTALL)
    return "\n\n".join(code_blocks).strip()
 
# Keep your existing run_crew_pipeline function, but with improvements
def run_crew_pipeline(query: str, formatted_result: str) -> dict:
    """Run the CrewAI pipeline to generate code based on the query and API results."""
    
    # Initialize the LLM
    try:
        llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
    except Exception as e:
        print(f"Error initializing LLM: {str(e)}")
        return {"error": f"LLM initialization failed: {str(e)}"}
    
    # 🔹 Improved Agents
    planner = Agent(
        role="System Architect",
        goal="Design a complete architecture using appropriate Sikka APIs",
        backstory="You're a senior architect with expertise in API integration and system design.",
        llm=llm,
    )
 
    backend_dev = Agent(
        role="Backend Developer",
        goal="Write production-quality FastAPI backend code using selected Sikka APIs",
        backstory="Expert in Python, REST APIs, and backend integration with proper error handling.",
        llm=llm,
    )
 
    frontend_dev = Agent(
        role="Frontend Developer",
        goal="Create a polished React frontend that interacts seamlessly with the backend",
        backstory="Frontend expert focused on React, state management, and clean UI design.",
        llm=llm,
    )
 
    # 🔹 Improved Tasks
    planner_task = Task(
        description=f"""
The user prompt is: \"{query}\"
 
Here are the top relevant Sikka APIs:
 
{formatted_result}
 
Your job:
1. Analyze the requirements carefully
2. Choose the most appropriate Sikka APIs to use
3. Design a comprehensive solution architecture
4. Write detailed guidance for the Backend and Frontend Developers
""",
        expected_output="A detailed architecture plan with specific API selections and implementation guidance.",
        agent=planner,
    )
 
    backend_task = Task(
        description="""
Using the planner's architecture:
1. Implement complete FastAPI routes for the selected Sikka APIs
2. Include proper request/response models
3. Add comprehensive error handling
4. Ensure security best practices are followed
5. Document all endpoints
 
Provide complete, production-ready code with all necessary imports and setup.
""",
        expected_output="Complete FastAPI backend code wrapped in a code block labeled ```FastAPI```.",
        agent=backend_dev,
    )
 
    frontend_task = Task(
        description="""
Using the planner's architecture:
1. Create a complete React component that interfaces with the backend
2. Implement proper form handling and validation
3. Add loading states and error handling
4. Design a clean and intuitive user interface
5. Ensure all API interactions are properly implemented
 
Provide a complete React implementation, not just snippets.
""",
        expected_output="Complete React component with clean UI and API integration.",
        agent=frontend_dev,
    )
 
    # 🔹 Run the crew with better error handling
    try:
        crew = Crew(
            agents=[planner, backend_dev, frontend_dev],
            tasks=[planner_task, backend_task, frontend_task],
            verbose=True,
        )
 
        final_output = crew.kickoff()
 
        # Extract code with improved function
        backend_code = extract_code(str(final_output), "FastAPI")
        frontend_code = extract_code(str(final_output), "React")
 
        return {
            "raw_output": str(final_output),
            "backend_code": backend_code if backend_code.strip() else None,
            "frontend_code": frontend_code if frontend_code.strip() else None
        }
    except Exception as e:
        print(f"Error in crew execution: {str(e)}")
        return {
            "error": str(e),
            "raw_output": None,
            "backend_code": None,
            "frontend_code": None
        }
 
# For manual testing
if __name__ == "__main__":
    query = "Make a payment method for a patient"
    api_matches = search_sikka_api(query, k=5)
    formatted = format_api_results(api_matches)
    result = run_crew_pipeline(query, formatted)
 
    if "error" in result and result["error"]:
        print(f"\n❌ Error: {result['error']}")
    else:
        if result["backend_code"]:
            print("\n✅ Backend Code Preview:\n", result["backend_code"][:200] + "..." if len(result["backend_code"]) > 200 else result["backend_code"])
        if result["frontend_code"]:
            print("\n✅ Frontend Code Preview:\n", result["frontend_code"][:200] + "..." if len(result["frontend_code"]) > 200 else result["frontend_code"])